In [1]:
import pandas as pd
import numpy as np

In [2]:
#read the files
namebasics = pd.read_csv('name.basics.tsv', sep='\t')
akas = pd.read_csv('title.akas.tsv', sep='\t')
titlebasics = pd.read_csv('title.basics.tsv', sep='\t')
principal = pd.read_csv('title.principals.tsv', sep='\t')
rating = pd.read_csv('title.ratings.tsv', sep='\t')

/var/folders/5m/lwhv14kx1ms95ccr8yphs_qr0000gn/T/ipykernel_63372/2769637842.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  akas = pd.read_csv('title.akas.tsv', sep='\t')
/var/folders/5m/lwhv14kx1ms95ccr8yphs_qr0000gn/T/ipykernel_63372/2769637842.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  titlebasics = pd.read_csv('title.basics.tsv', sep='\t')


In [3]:
#selecting useful columns

In [4]:
namebasics = namebasics.loc[:, 'nconst':'primaryName']

In [5]:
akas = akas.iloc[:, [0,2,3,7]]
akas = akas.query('isOriginalTitle == 0') # no region info if isOringinalTitle = 1
akas = akas.rename(columns={"titleId": "tconst"})

In [6]:
titlebasics = titlebasics.loc[:, ['tconst', 'titleType', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres']]

In [7]:
principal = principal.query('category == "actor" | category == "self" | category == "actress"')
principal = principal.loc[:, ['tconst','nconst']]

In [8]:
rating = rating.loc[:, ['tconst', 'averageRating']]

In [9]:
# merge df
imdb = pd.merge(titlebasics, rating, how="left", on="tconst")
imdb = pd.merge(imdb, akas, how="outer", on="tconst")
imdb = pd.merge(imdb, principal, how="outer", on="tconst")
imdb = pd.merge(imdb, namebasics, how="left", on="nconst")

In [10]:
# select movie type only
imdb = imdb.query('titleType == "movie"')

In [11]:
# drop na and unuseful columns
imdb = imdb.replace(r'\\N', np.NaN, regex=True)
#imdb = imdb.dropna()
imdb = imdb.drop(columns=['titleType', 'title', 'isOriginalTitle', 'nconst'])

In [12]:
# change year and runtime to int data type, genres to string
imdb['startYear'] = imdb['startYear'].astype('float')
imdb['runtimeMinutes'] = imdb['runtimeMinutes'].astype('float')
imdb['genres'] = imdb['genres'].astype('string')

In [13]:
# explode genres
imdb = imdb.drop('genres', axis=1).join(imdb['genres'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('genres'))

In [14]:
imdb = imdb.query('startYear >2010 & startYear < 2021')

In [15]:
imdb = imdb.query('genres == "Action" | genres == "Crime" | genres == "Horror" | genres == "Mystery" | genres == "Romance" | genres == "Sci-Fi" | genres == "Thriller"')

In [16]:
imdb.to_csv("imdb.csv")